In [1]:
#%pip install equinox openmdao[all]

In [2]:
import equinox as eqx
from util import generate_base_model, density, dyn_viscousity
import jax
import jax.numpy as jnp

In [3]:
base_model = generate_base_model()

lift_model = eqx.tree_deserialise_leaves(f"models/lift_surrogate.eqx", base_model)
drag_model = eqx.tree_deserialise_leaves(f"models/drag_surrogate.eqx", base_model)

In [4]:
from optimizer import Optimizer

In [5]:
opt = Optimizer()

In [6]:
opt.load_surrogates(lift_model, drag_model)

In [7]:
opt.define_material(
    E=69e9,
    rho=2700,
    yield_strength=241e6,
    shear_strength=207e6
)

In [8]:
altitude = 10_000

results = opt.solve_wing(
    lift_goal=4e6,
    safety_factor=0.75,
    v_infty=300,
    rho=density(altitude),
    mu=dyn_viscousity(altitude),
    alpha_geo=jnp.deg2rad(jnp.float32(5))
)

Optimizing planform


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 115456.37806280811
            Iterations: 30
            Function evaluations: 75
            Gradient evaluations: 28
Optimization Complete
-----------------------------------
Cl_0 goal: [0.06122096]
Optimizing airfoil
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0038773777694477017
            Iterations: 26
            Function evaluations: 35
            Gradient evaluations: 26
Optimization Complete
-----------------------------------


In [9]:
results

{'parameters': {'B': array([1.70974407]),
  'T': array([0.16461884]),
  'P': array([2.82824866]),
  'C': array([9.68793933e-20]),
  'E': array([0.96024825]),
  'R': array([0.01725691]),
  'b': array([80.12051725]),
  'c': array([5.80708666]),
  'AR': array([13.79702455])},
 'aerodynamics': {'L': Array(4011409.50097191, dtype=float64),
  'D': Array(82506.35639812, dtype=float64),
  'Cl_0': array([0.06122096]),
  'alpha_0': Array([-0.55826823], dtype=float64)},
 'structure': {'normal': 0.7500603604980083,
  'shear': 0.4286526687216571,
  'flange_w': array([0.25]),
  'flange_h': array([0.03796362]),
  'web_w': array([0.005]),
  'main_x': array([0.25]),
  'rear_x': array([0.75])}}